In [1]:
%load_ext line_profiler

In [2]:
import requests
import pandas as pd

import re
import os

In [3]:
URL = "https://vi.wikipedia.org/w/api.php"
PARAMS_CAT = {
    "action": "query",
    "cmlimit": "100",
    "list": "categorymembers",
    "format": "json",
    
    "cmtitle": "",
    "cmtype": ""
}

PARAMS_WIKITEXT = {
    "action": "parse",
    "prop": "wikitext",
    "formatversion": 2,
    "format": "json",
    
    "page": ""
}

PARAMS_HTML = {
    "action": "parse",
    "formatversion": 2,
    "format": "json",
    
    "page": ""
}

SESSION = requests.Session()

In [23]:
tempRecurse = []

def recurseCat(S, catdict, title, level, maxlevel):
        # RECURSIVE
    PARAMS_CAT['cmtitle'] = title
    R = S.get(url=URL, params=PARAMS_CAT)
    DATA = R.json()
    # print(DATA)
    PAGES = DATA['query']['categorymembers']

    index = 1
    for page in PAGES:
        childtitle = page['title']
        if childtitle in tempRecurse:
            print("Already exist");
            continue

        tempRecurse.append(childtitle)
        catdict.append({"cats": childtitle, "parent": title})
        print("- "*level, index, childtitle)
        
        if (level < maxlevel):
            recurseCat(S, catdict, childtitle, level+1, maxlevel)
        else:
            print("Max level reached")
        index += 1

def initRecurseCat(S, catdict, title, maxlevel=5):
    tempRecurse.clear()
    PARAMS_CAT['cmtype'] = "subcat"
    
    recurseCat(S, catdict, title, 0, maxlevel)

In [5]:
import mwparserfromhell
from bs4 import BeautifulSoup

DEF_TEMPLATE_NAMES = "Thông tin|Infobox"

def template_infobox(tpl):
    return re.search(DEF_TEMPLATE_NAMES, tpl.name.strip_code(), re.IGNORECASE)

def title_to_infoboxes_names(title):
    PARAMS_WIKITEXT['page'] = title
    response = SESSION.get(url=URL, params=PARAMS_WIKITEXT)
    data = response.json()['parse']['wikitext']
        
    wikicode = mwparserfromhell.parse(data)
    templates = wikicode.filter_templates(matches=template_infobox)
    
    template_names = list(map(lambda x: str(x.name), templates))
    
    result_temp = ''.join(template_names)
    
    return result_temp

In [6]:
def title_to_infoboxes_wikicode(title):
    PARAMS_WIKITEXT['page'] = title
    response = SESSION.get(url=URL, params=PARAMS_WIKITEXT)
    data = response.json()['parse']['wikitext']
        
    wikicode = mwparserfromhell.parse(data)
    templates = wikicode.filter_templates(matches=template_infobox)
    
    # Find the infobox template
    if not templates:
        print("Template not found: " + title)
        return {}
    
    # Extract the key-value pairs from the infobox
    attrlist = {}
    for param in templates[0].params:
        val = str(param.value).strip()
        if bool(val):
            attrlist[str(param.name).strip()] = val # Key
    
    attrlist['title'] = title
    return attrlist

In [7]:
def title_to_infoboxes_html(title):
    PARAMS_HTML['page'] = title
    response = SESSION.get(url=URL, params=PARAMS_HTML)
    data = response.json()['parse']['text']
        
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(data, "html.parser")

    # Find the infobox template and extract its contents
    infobox = soup.find("table", {"class": "infobox"})
    if not infobox:
        print("Infobox not found: " + title)
        return {}
        
    rows = infobox.find_all("tr")

    # Convert the infobox contents into a Pandas DataFrame
    
    # err_key = []; err_val = [];
    
    attrlist = {}
    for row in rows:
        cell_key = row.find_all(["th"])
        cell_val = row.find_all(["td"])
        # has_table = row.find("table")
        if len(cell_key) == 1 and len(cell_val) == 1: # Deal with tables later
            key = cell_key[0].get_text().strip()
            value = cell_val[0].get_text().strip()
            attrlist[key] = value
#         else:
#             # Determine the maximum length of the lists
#             max_length = max(len(cell_key), len(cell_val))
#             for i in range(max_length):
#                 if i < len(cell_key): err_key.append(cell_key[i].get_text().strip())
#                 else: err_key.append(None);
                    
#                 if i < len(cell_val): err_val.append(cell_val[i].get_text().strip())
#                 else: err_val.append(None)
    
#     display(pd.DataFrame({'ERROR KEY': err_key, 'ERROR VAL': err_val}))
    attrlist['title'] = title
    return attrlist

In [8]:
url = 'https://vi.wikipedia.org/w/api.php?action=parse&prop=wikitext&format=json&page='
current = "Nhà Trần"

%lprun -f title_to_infoboxes_names title_to_infoboxes_names(current)

Timer unit: 1e-07 s

Total time: 3.19638 s
File: C:\Users\Legion\AppData\Local\Temp\ipykernel_29008\3912060920.py
Function: title_to_infoboxes_names at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def title_to_infoboxes_names(title):
    10         1         36.0     36.0      0.0      PARAMS_WIKITEXT['page'] = title
    11         1   28025278.0 28025278.0     87.7      response = SESSION.get(url=URL, params=PARAMS_WIKITEXT)
    12         1      17101.0  17101.0      0.1      data = response.json()['parse']['wikitext']
    13                                                   
    14         1    3356307.0 3356307.0     10.5      wikicode = mwparserfromhell.parse(data)
    15         1     564712.0 564712.0      1.8      templates = wikicode.filter_templates(matches=template_infobox)
    16                                               
    17         1        343.0    343.0      0.0      template_names = list(

In [9]:
%lprun -f title_to_infoboxes_wikicode title_to_infoboxes_wikicode(current)

Timer unit: 1e-07 s

Total time: 1.31913 s
File: C:\Users\Legion\AppData\Local\Temp\ipykernel_29008\2458799018.py
Function: title_to_infoboxes_wikicode at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def title_to_infoboxes_wikicode(title):
     2         1         19.0     19.0      0.0      PARAMS_WIKITEXT['page'] = title
     3         1    8839492.0 8839492.0     67.0      response = SESSION.get(url=URL, params=PARAMS_WIKITEXT)
     4         1      13193.0  13193.0      0.1      data = response.json()['parse']['wikitext']
     5                                                   
     6         1    3736227.0 3736227.0     28.3      wikicode = mwparserfromhell.parse(data)
     7         1     583320.0 583320.0      4.4      templates = wikicode.filter_templates(matches=template_infobox)
     8                                               
     9                                               # Find the infobo

In [10]:
%lprun -f title_to_infoboxes_html title_to_infoboxes_html(current)

Timer unit: 1e-07 s

Total time: 2.19308 s
File: C:\Users\Legion\AppData\Local\Temp\ipykernel_29008\4170776269.py
Function: title_to_infoboxes_html at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def title_to_infoboxes_html(title):
     2         1         16.0     16.0      0.0      PARAMS_HTML['page'] = title
     3         1   12468977.0 12468977.0     56.9      response = SESSION.get(url=URL, params=PARAMS_HTML)
     4         1      47036.0  47036.0      0.2      data = response.json()['parse']['text']
     5                                                   
     6                                               # Parse the HTML content using BeautifulSoup
     7         1    8989290.0 8989290.0     41.0      soup = BeautifulSoup(data, "html.parser")
     8                                           
     9                                               # Find the infobox template and extract its contents
    

In [11]:
def getAllPages(S, SET_OF_CATS):
    data = []
    
    i = 0
    for cat in SET_OF_CATS:
          # GET ALL PAGES
        PARAMS_CAT['cmtitle'] = cat
        PARAMS_CAT['cmtype'] = "page"
        DATA = S.get(url=URL, params=PARAMS_CAT).json()
        
        PAGES = DATA['query']['categorymembers']
        for page in PAGES:
            page_title = page['title']
            
            if "Bản mẫu:" in page_title: # Skip bản mẫu
                continue;
                
            thispage = {"page": page_title, "cat": cat }
            data.append(thispage)
            print(thispage)
            
        i += 1
        print(f'Progress: {i}', end='\r')
            
    return pd.DataFrame(data)

## TESTING & DEFINITIONS END HERE

In [25]:
toptitle = "Thể_loại:Lễ_hội_Việt_Nam"

In [28]:
if not os.path.isfile('Categories.csv'):
    new_cat_dict = [{'cats': toptitle, 'parent': toptitle }]

    initRecurseCat(SESSION, new_cat_dict, toptitle, maxlevel = 3) # Already removed duplicates
    print("\nThat's all")
    
    df1 = pd.DataFrame(new_cat_dict)
    df1.to_csv('Unfiltered_Categories.csv', index=False) # Added for checking coverage


    regex = re.compile('lễ|hội|:Tết', re.IGNORECASE)

    print(df1.shape); df1 = df1[df1['cats'].str.contains(regex)].reset_index(drop=True)
    print(df1.shape);
    
    df1.to_csv('Categories.csv', index=False)
else:
    df1 = pd.read_csv('Categories.csv')
    
df1

 1 Thể loại:Ngày lễ Việt Nam
-  1 Thể loại:Lễ hội Việt Nam
Already exist
- -  1 Thể loại:Lễ hội các dân tộc Việt Nam
- - -  1 Thể loại:Lễ hội của người Ba Na
Max level reached
- - -  2 Thể loại:Lễ hội của người Chăm
Max level reached
- - -  3 Thể loại:Lễ hội của người Chơ Ro
Max level reached
- - -  4 Thể loại:Lễ hội của người Cống
Max level reached
- - -  5 Thể loại:Lễ hội của người Dao
Max level reached
- - -  6 Thể loại:Lễ hội của người Ê Đê
Max level reached
- - -  7 Thể loại:Lễ hội của người Giáy
Max level reached
- - -  8 Thể loại:Lễ hội của người H'Mông
Max level reached
- - -  9 Thể loại:Lễ hội của người Hà Nhì
Max level reached
- - -  10 Thể loại:Lễ hội của người Hoa
Max level reached
- - -  11 Thể loại:Lễ hội của người Kháng
Max level reached
- - -  12 Thể loại:Lễ hội của người Khmer
Max level reached
- - -  13 Thể loại:Lễ hội của người Khơ Mú
Max level reached
- - -  14 Thể loại:Lễ hội của người La Ha
Max level reached
- - -  15 Thể loại:Lễ hội của người Lô Lô
Max level reac

,cats,parent
0,Thể_loại:Lễ_hội_Việt_Nam,Thể_loại:Lễ_hội_Việt_Nam
1,Thể loại:Ngày lễ Việt Nam,Thể_loại:Lễ_hội_Việt_Nam
2,Thể loại:Lễ hội Việt Nam,Thể loại:Ngày lễ Việt Nam
3,Thể loại:Lễ hội các dân tộc Việt Nam,Thể loại:Lễ hội Việt Nam
4,Thể loại:Lễ hội của người Ba Na,Thể loại:Lễ hội các dân tộc Việt Nam
5,Thể loại:Lễ hội của người Chăm,Thể loại:Lễ hội các dân tộc Việt Nam
6,Thể loại:Lễ hội của người Chơ Ro,Thể loại:Lễ hội các dân tộc Việt Nam
7,Thể loại:Lễ hội của người Cống,Thể loại:Lễ hội các dân tộc Việt Nam
8,Thể loại:Lễ hội của người Dao,Thể loại:Lễ hội các dân tộc Việt Nam
9,Thể loại:Lễ hội của người Ê Đê,Thể loại:Lễ hội các dân tộc Việt Nam


In [46]:
if not os.path.isfile('Pages.csv'):
    all_pages = getAllPages(SESSION, df1.cats)
    
    print(all_pages.shape); all_nodup = all_pages.drop_duplicates(subset=['page'], keep='first').copy()
    
    words_place = [
        'Lễ',
        'Hội'
    ]

    regex = re.compile('|'.join(words_place))
    all_nodup.loc[all_nodup['page'].str.contains(regex), 'filter'] = 1
    all_filter = all_nodup[all_nodup['filter'] == 1].drop('filter', axis=1)
    
    all_nodup.to_csv('Unfiltered_Pages.csv', index=False)
    all_filter.to_csv('Pages.csv', index=False)
else:
    all_filter = pd.read_csv('Pages.csv')

all_filter.sort_values(by='page')

{'page': 'Lễ hội Việt Nam', 'cat': 'Thể_loại:Lễ_hội_Việt_Nam'}
{'page': 'Bơi Trải', 'cat': 'Thể_loại:Lễ_hội_Việt_Nam'}
{'page': 'Chợ Âm Dương', 'cat': 'Thể_loại:Lễ_hội_Việt_Nam'}
{'page': 'Giỗ Tổ Hùng Vương', 'cat': 'Thể_loại:Lễ_hội_Việt_Nam'}
{'page': 'Lễ mừng lúa mới', 'cat': 'Thể_loại:Lễ_hội_Việt_Nam'}
{'page': 'Lễ hội chọi trâu', 'cat': 'Thể_loại:Lễ_hội_Việt_Nam'}
{'page': 'Lễ hội chùa Bái Đính', 'cat': 'Thể_loại:Lễ_hội_Việt_Nam'}
{'page': 'Lễ hội đền Hùng', 'cat': 'Thể_loại:Lễ_hội_Việt_Nam'}
{'page': 'Lễ hội đền Thái Vi', 'cat': 'Thể_loại:Lễ_hội_Việt_Nam'}
{'page': 'Lễ hội pháo hoa quốc tế Đà Nẵng', 'cat': 'Thể_loại:Lễ_hội_Việt_Nam'}
{'page': 'Múa kiếm', 'cat': 'Thể_loại:Lễ_hội_Việt_Nam'}
{'page': 'Lễ hội nghinh Ông', 'cat': 'Thể_loại:Lễ_hội_Việt_Nam'}
{'page': 'Năm Du lịch quốc gia 2020', 'cat': 'Thể_loại:Lễ_hội_Việt_Nam'}
{'page': 'Năm Du lịch quốc gia 2021', 'cat': 'Thể_loại:Lễ_hội_Việt_Nam'}
{'page': 'Núi Voi (Hải Phòng)', 'cat': 'Thể_loại:Lễ_hội_Việt_Nam'}
{'page': 'Phủ Dầy',

,page,cat
146,Hội Gióng,Thể loại:Lễ hội Hà Nội
139,Hội Lim,Thể loại:Lễ hội Bắc Ninh
175,Hội Yến Diêu Trì Cung,Thể loại:Lễ hội Tây Ninh
171,Hội chọi trâu Phù Ninh,Thể loại:Lễ hội Phú Thọ
138,Hội làng Long Khám,Thể loại:Lễ hội Bắc Ninh
...,...,...
166,Lễ hội đền Nguyễn Công Trứ,Thể loại:Lễ hội Ninh Bình
8,Lễ hội đền Thái Vi,Thể_loại:Lễ_hội_Việt_Nam
173,Lễ khao lề thế lính,Thể loại:Lễ hội Quảng Ngãi
73,Lễ mở cửa tháp (Chăm),Thể loại:Lễ hội của người Chăm


In [47]:
from tqdm import tqdm

In [48]:
DEF_TEMPLATE_NAMES = "Thông tin|Infobox"

In [49]:
tqdm.pandas()
all_filter.loc[:,'infoboxes'] = all_filter['page'].progress_apply(title_to_infoboxes_names)

100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [00:46<00:00,  2.22it/s]


In [51]:
all_filter.to_csv('Pages_Infoboxes.csv', index=False)

In [141]:
tqdm.pandas()
wikicode_list = all_filter['page'].progress_apply(title_to_infoboxes_wikicode)

 35%|█████████████████████████████                                                      | 7/20 [00:03<00:06,  1.90it/s]

Template not found: Nhà Hậu Lê


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [00:11<00:00,  1.60it/s]

Template not found: Kênh Nhà Lê


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:13<00:00,  1.50it/s]


In [142]:
df = pd.json_normalize(wikicode_list)
df = df.dropna(subset=['title'])
print(df.shape)

(18, 118)


In [143]:
# Sort column by most popular
counts = df.count().reset_index()
counts.columns = ['col', 'count']
counts = counts.sort_values(by='count',ascending=False)

df = df.reindex(counts['col'], axis=1)
df

col,common_languages,common_name,conventional_long_name,title,government_type,year_start,year_end,p1,s1,capital,...,flag_p2,flag_s1,flag_s2,s3,flag_s3,s4,flag_s4,s5,flag_s5,footnotes
0,[[Tiếng Hán|Hán ngữ trung đại]],Đinh triều,Đại Cồ Việt,Nhà Đinh,[[Quân chủ]],968,980,Nhà Ngô,Nhà Tiền Lê,[[Hoa Lư]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[[Tiếng Việt]]<br>[[Tiếng Hán]],Nhà Trần,Nhà Hậu Trần<br>茹後陳,Nhà Hậu Trần,[[Quân chủ chuyên chế]],1407,1414,Kỷ thuộc Minh,Kỷ thuộc Minh,[[Mô Độ]] (1407 - 1409)<br>[[Bình Than]] (1409...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[[Tiếng Việt|Việt ngữ trung đại]],Nhà Lê sơ,Đại Việt,Nhà Lê sơ,[[Quân chủ tuyệt đối]],[[1428]],[[1527]],Bắc thuộc lần 4,Nhà Mạc,[[Hà Nội|Đông Kinh]]<br/><small>(東京 [[1428]] -...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[[Tiếng Việt|Tiếng Việt trung đại]],Hậu Lê triều,Đại Việt,Nhà Lê trung hưng,[[Chế độ quân chủ]],1533,1789,Nhà Mạc,Nhà Tây Sơn,[[Kinh đô Vạn Lại - An Trường|Vạn Lại]] (tạm t...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[[Tiếng Việt và tiếng Hán trung cổ|Việt ngữ và...,Ngô,Tĩnh Hải quân,Nhà Ngô,Quân chủ,939,965,Bắc thuộc lần 3,Nhà Đinh,[[Cổ Loa]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,[[Tiếng Việt|Tiếng Việt trung đại]],Triều Hồ,Đại Ngu,Nhà Hồ,Quân chủ,1400,1407,Nhà Trần,Bắc thuộc lần 4,[[Thành Tây Đô|Tây Đô]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,[[Tiếng Việt|Tiếng Việt trung đại]],nhà Lý,Nhà Lý,Nhà Lý,Quân chủ,1009,1225,Nhà Tiền Lê,Nhà Trần,[[Hoa Lư]] (1009 - 1010)<br>[[Thăng Long]] (10...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,[[Tiếng Việt]],Nhà Mạc,<!-- See the talk page before editing please. -->,Nhà Mạc,Quân chủ,1527,1677,Nhà Lê sơ,Nhà Lê trung hưng,[[Thăng Long|Đông Kinh]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,[[Tiếng Việt]]{{br}}[[Văn ngôn|Văn ngôn Trung ...,Triều Nguyễn,Đại Việt <small>(1802–1804)</small><br />Việt ...,Nhà Nguyễn,[[Quân chủ chuyên chế]],1802,1945,Nhà Tây Sơn,Nam Kỳ{{!}}{{small|'''1862:'''}}<br />Nam Kỳ,[[Phú Xuân]] (nay là [[Huế]]),...,Seal of Nguyễn Lords.svg,Flag of France (1794-1815).svg,Flag of Central Vietnam (1885-1890).svg,Bắc Kỳ{{!}}Bắc Kỳ,Flag of Central Vietnam (1885-1890).svg,Việt Nam Dân chủ Cộng hòa{{!}}{{small|'''1945:...,Flag of North Vietnam 1945-1955.svg,Đông Dương thuộc Pháp,Flag of France (1794-1815).svg,NaN
10,[[Tiếng Việt|Tiếng Việt cổ]]\n\n[[Tiếng Trung|...,Vạn Xuân,Vạn Xuân,Nhà Tiền Lý,Chế độ quân chủ,544,602,Bắc thuộc lần 2,Bắc thuộc lần 3,[[Long Biên]],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
df.to_csv('wikicodeparse.csv', index=False)
df.shape

(18, 118)

In [60]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# HTML

In [196]:
tqdm.pandas()
html_list = all_filter['page'].progress_apply(title_to_infoboxes_html)

 35%|█████████████████████████████                                                      | 7/20 [00:06<00:12,  1.08it/s]

Infobox not found: Nhà Hậu Lê


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [00:20<00:00,  1.08it/s]

Infobox not found: Kênh Nhà Lê


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:22<00:00,  1.13s/it]


In [197]:
df = pd.json_normalize(html_list)
df = df.dropna(subset=['title'])
print(df.shape)

(18, 170)


In [198]:
# Sort column by most popular
counts = df.count().reset_index()
counts.columns = ['col', 'count']
counts = counts.sort_values(by='count',ascending=False)

df = df.reindex(counts['col'], axis=1)
df

col,title,Thủ đô,Đơn vị tiền tệ,Ngôn ngữ thông dụng,Chính phủ,Lịch sử,Tôn giáo chính,Hoàng đế,Vị thế,Hiện nay là một phần của,...,• 1638-1677,• Mạc Thái Tổ soán ngôi nhà Lê sơ,• Nhà Lê trung hưng tái lập ở Ai Lao,"• Đông Kinh thất thủ, Mạc Mậu Hợp bị giết",• Tàn dư họ Mạc chấm dứt,• 1540,• 1802–1820 (đầu),• 1884–1885 (cuối cùng tronggiai đoạn độc lập),• 1926–1945 (cuối cùng),• Hồ Quý Ly soán ngôi Nhà Trần
0,Nhà Đinh,Hoa Lư,Tiền xu,Hán ngữ trung đại,Quân chủ,,"Phật giáo, Đạo giáo,...",,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Nhà Hậu Trần,Mô Độ (1407 - 1409)Bình Than (1409 - 1414),Tiền xu,Tiếng ViệtTiếng Hán,Quân chủ chuyên chế,,Tam giáo quy nguyên,NaN,Đế quốc,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nhà Lê sơ,Đông Kinh(東京 1428 - 1527),Tiền xu,Việt ngữ trung đại,Quân chủ tuyệt đối,,"Phật giáo, Nho giáo, Tín ngưỡng dân gian",,Đế quốc,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Nhà Lê trung hưng,Vạn Lại (tạm thời)(1533–1597)Đông Kinh(1597–1789),Tiền xu,Tiếng Việt trung đại,Chế độ quân chủ,,Nho giáo Phật giáo Đạo giáo Công giáo Tín ngưỡ...,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Nhà Ngô,Cổ Loa,Tiền xu,Việt ngữ và Hán ngữ trung đại,Quân chủ,,"Phật giáo, Đạo giáo, Cảnh giáo[1]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Nhà Hồ,"Tây Đô20°4′B 105°36′Đ﻿ / ﻿20,067°B 105,6°Đ﻿ / ...","Thông bảo hội sao (tiền giấy), Thánh Nguyên th...",Tiếng Việt trung đại,Quân chủ,,"Phật giáo, Đạo giáo, Tín ngưỡng Việt Nam",,Đế quốc,Việt Nam Trung Quốc Lào,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Nhà Lý,Hoa Lư (1009 - 1010)Thăng Long (1010 - 1225),Tiền xu,Tiếng Việt trung đại,Quân chủ,,Phật giáo,,NaN,Việt Nam Trung Quốc Lào,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Nhà Mạc,Đông Kinh,Tiền xu,Tiếng Việt,Quân chủ,,Tam giáo quy nguyên,,Đế quốc,NaN,...,Mạc Kính Vũ,1527,1533,1592,1677,2.000.000,NaN,NaN,NaN,NaN
9,Nhà Nguyễn,"Phú Xuân (nay là Huế)16°28′B 107°36′Đ﻿ / ﻿16,4...",Quan,Tiếng ViệtVăn ngôn Trung Quốc[1][2][Ghi chú 1]...,Quân chủ chuyên chế,,"Nho giáo, Phật giáo, Công giáo, Tín ngưỡng dân...",,Đế quốc nội địa chịu sự triều cống cho Trung Q...,Campuchia Lào Trung Quốc Việt Nam,...,NaN,NaN,NaN,NaN,NaN,NaN,Gia Long,Hàm Nghi,Bảo Đại,NaN
10,Nhà Tiền Lý,Long Biên,Văn,Tiếng Việt cổ\nTiếng Hán trung đại,Chế độ quân chủ,,NaN,NaN,NaN,Việt Nam Trung Quốc Lào,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [199]:
df.to_csv('htmlparse.csv', index=False)
df.shape

(18, 170)

In [60]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)